In [1]:
import argparse
# Define functions to be used
def parse_arguments(args):
    # Define the command-line arguments
# Create an argument parser
    parser = argparse.ArgumentParser(description="Script description")

    # Add command-line arguments
    # Arguments for online use 
    parser.add_argument("--username", default="", help="Online account name or omit for offline use")
    parser.add_argument("--type", default="deck", choices=["deck", "fuseddeck"], help="Decktype for user collection , default=deck")
    parser.add_argument("--id", default="", help="Specific Deck ID from solforgefusion website")
    
    # Arguments for general use 
    # If both username and file is given, export deckbase to file.json 
    # If only file is given import deckbase from file.json 
    parser.add_argument("--filename",  default=None,  help="Offline Deck Database Name")
    parser.add_argument("--synergies", default=None, help="CSV Filename for synergy lookup")    
    parser.add_argument("--offline", default=None, help="Offline use only")    
    parser.add_argument("--mode", default='insert', help="Mode: insert, update, refresh")    

    # Arguments for Evaluation
    
    parser.add_argument("--eval", nargs='?', const=True, action="store",  help="Evaluate possible fusions. Optional filename for .csv export")    
    parser.add_argument("--graph", action="store_true",  help="Create Graph '.gefx'")
    parser.add_argument("--filter", default=None, help="Filter by card names. Syntax: \"<cardname>+'<card name>'-<cardname>\" + = AND, - = OR ")
    parser.add_argument("--select_pairs", action="store_true", help="Select top pairs")
    
    # Parse the command-line arguments
    return parser.parse_args(args)
    



In [4]:
import soldb 
import os

# Define Variables
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

username = 'tmind'
param_eval = 'tmind'

arguments = ["--username" , username,
             "--eval" , param_eval,
             "--mode", "update"    ]

args = parse_arguments(arguments)

soldb.main(args)



Requesting Data from Website: tmind,deck,


Fetching Data: 100%|██████████| 137/137 [00:00<?, ?it/s]


Requesting Data from Website: tmind,fuseddeck,


  0%|          | 0/18632 [00:00<?, ?it/s]